In [78]:
import os
import openai
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import time


# Connect to ChatGpt Api

In [79]:
openai.organization = "org-RuJeUUCVE9l16B6VH1CFTTJn"
openai.api_key = os.getenv("CHATGPTKEY")
openai.Model.list()

<OpenAIObject list at 0x146387180> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampl

## Check if the connection and saving is working


In [80]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-79x5sd8ykDYb9g21DmgLrtNiQQWK1 at 0x132779b80> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played at Globe Life Field in Arlington, Texas.",
        "role": "assistant"
      }
    }
  ],
  "created": 1682606464,
  "id": "chatcmpl-79x5sd8ykDYb9g21DmgLrtNiQQWK1",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 57,
    "total_tokens": 74
  }
}

## Start testing

In [81]:
# open our test dataframe from the csv
df = pd.read_csv('data/test/curated_benchmark_aaron.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,...,specTableContent_right,cluster_id_right,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,Aaron
0,0,1349,45334125,NaN,Pen Drive SanDisk 64GB Cruzer Glide USB 2.0,NaN,"9,86",EUR,NaN,763994,...,NaN,279041,45334125#9787254,0,True,"[3.7204918861, -4.1918787956]",0,[0.0],0,1.0
1,1,1056,7567487,NaN,PEN DRIVE 16GB KINGSTON USB 3.0 DTIG4/16GB,"PEN DRIVE 16GB KINGSTON USB 3.0 DTIG4/16GB DataTraveler Chiavetta USB 16 GB, Bianco Blu",Apr 84,EUR,NaN,1915191,...,NaN,1915191,7567487#83075662,1,False,"[-4.3719258308, 4.6600718497999996]",1,[1.0],1,1.0
2,2,244,68806630,Swiss Military Hanowa,Rel√≥gio Swiss Military Hanowa06-5161.2.04.003 Flagship,Steel Swiss Made Watch with Date,249.00,EUR,NaN,2682919,...,NaN,2080188,68806630#60576778,0,True,"[3.7885432243, -4.2523846626]",0,[0.0],0,1.0
3,5,3154,33138819,NaN,Baterie APC Battery kit APCRBC110,"Baterie APC Battery kit APCRBC110, Baterie, pro UPS, kit APCRBC110 pro BE550G-CP, BE550G-FR, BR550GI",1399.70,CZK,NaN,1157842,...,NaN,849634,33138819#8401,0,True,"[3.8125219345, -4.2747883797]",0,[0.0],0,1.0
4,6,3963,81415527,NaN,2MP 25X Network IR PTZ Camera,"1/2.8\"" progressive scan CMOSUp to 1920 √ó 1080@30fps resolutionMin. illumination: Color: 0.005 Lux @(F1.6, AGC ON) B/W: 0.001 Lux @(F1.6, AGC ON) 0 Lux with IR25√ó optical zoom, 16√ó digital zoomWDR, HLC, BLC, 3D DNR, Defog, EIS, Regional Exposure, Regional FocusUp to 150 m IR distance24 VAC & ...","Incl. BTW‚Ç¨1.035,76",EUR,NaN,2694265,...,NaN,2694265,81415527#46810114,1,False,"[-4.1793274879, 4.4736194611]",1,[1.0],1,1.0


In [82]:
# create a new column for the chatbot response
df['chatbot_response'] = ''
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [97]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = [
        {"role": "system", "content": "Do the following two product decriptions match. Answer with yes or no."},
        {"role": "user",
            "content": f" product 1: {row['title_left']}, product 2: {row['title_right']}"},
    ]
    error = ""
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=2,
            )
            # if the above call succeeds, break out of the loop and continue
            break
        except Exception as e:
            error = e
            print(f"Error: {e}")
            # if the call fails, wait for one minute before retrying
            time.sleep(60)

    # check if the response was set successfully
    if response:
        # your code here
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_response'] = response['choices'][0]['message']['content']
        df.at[index, 'chatbot_question'] = message
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_response'] = -1
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/100 [00:00<?, ?it/s]

Error: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e1a9848855f37871df85670cd3835812 in your message.)


In [98]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,...,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,Aaron,chatbot_response,chatbot_response_raw,chatbot_question,chatbot_response_clean
0,0,1349,45334125,NaN,Pen Drive SanDisk 64GB Cruzer Glide USB 2.0,NaN,"9,86",EUR,NaN,763994,...,True,"[3.7204918861, -4.1918787956]",0,[0.0],0,1.0,No.,"{'id': 'chatcmpl-79xJCbWRJW2u7RXz6KthCBMpufN0P', 'object': 'chat.completion', 'created': 1682607290, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 75, 'completion_tokens': 2, 'total_tokens': 77}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Pen Drive SanDisk 64GB Cruzer Glide USB 2.0, product 2: SanDisk 64GB SDIX30N-064G-GN6NN USB 3.0 Flash Drive'}]",1
1,1,1056,7567487,NaN,PEN DRIVE 16GB KINGSTON USB 3.0 DTIG4/16GB,"PEN DRIVE 16GB KINGSTON USB 3.0 DTIG4/16GB DataTraveler Chiavetta USB 16 GB, Bianco Blu",Apr 84,EUR,NaN,1915191,...,False,"[-4.3719258308, 4.6600718497999996]",1,[1.0],1,1.0,Yes.,"{'id': 'chatcmpl-79xJEbNsU6jB0LaSiCperK2RJb6FI', 'object': 'chat.completion', 'created': 1682607292, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 92, 'completion_tokens': 2, 'total_tokens': 94}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: PEN DRIVE 16GB KINGSTON USB 3.0 DTIG4/16GB, product 2: Kingston Technology DataTraveler G4 USB flash drive 16 GB USB Type-A 3.2 Gen 1 (3.1 Gen 1) Bla...",1
2,2,244,68806630,Swiss Military Hanowa,Rel√≥gio Swiss Military Hanowa06-5161.2.04.003 Flagship,Steel Swiss Made Watch with Date,249.00,EUR,NaN,2682919,...,True,"[3.7885432243, -4.2523846626]",0,[0.0],0,1.0,No.,"{'id': 'chatcmpl-79xJFycubDuPFjrZLs1FTOlpAqEvN', 'object': 'chat.completion', 'created': 1682607293, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 74, 'completion_tokens': 2, 'total_tokens': 76}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Rel√≥gio Swiss Military Hanowa06-5161.2.04.003 Flagship, product 2: Swiss Military Hanowa 06-4226.30.003.03'}]",0
3,5,3154,33138819,NaN,Baterie APC Battery kit APCRBC110,"Baterie APC Battery kit APCRBC110, Baterie, pro UPS, kit APCRBC110 pro BE550G-CP, BE550G-FR, BR550GI",1399.70,CZK,NaN,1157842,...,True,"[3.8125219345, -4.2747883797]",0,[0.0],0,1.0,No.,"{'id': 'chatcmpl-79xJGVQ6Rdq5aCPgR7wiS9Pi1VYjX', 'object': 'chat.completion', 'created': 1682607294, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 60, 'completion_tokens': 2, 'total_tokens': 62}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'system', 'content': 'Do the following two product decriptions match. Answer with yes or no.'}, {'role': 'user', 'content': ' product 1: Baterie APC Battery kit APCRBC110, product 2: APC Replacement Battery Cartridge #117 (APCRBC117)'}]",0
4,6,3963,81415527,NaN,2MP 25X Network IR PTZ Camera,"1/2.8\"" progressive scan CMOSUp to 1920 √ó 1080@30fps resolutionMin. illumination: Color: 0.005 Lux @(F1.6, AGC ON) B/W: 0.001 Lux @(F1.6, AGC ON) 0 Lux with IR25√ó optical zoom, 16√ó digital zoomWDR, HLC, BLC, 3D DNR, Defog, EIS, Regional Exposure, Regional FocusUp to 150 m IR distance24 VAC & ...","Incl. BTW‚Ç¨1.035,76",EUR,NaN,2694265,...,False,"[-4.1793274879, 4.4736194611]",1,[1.0],1,1.0,Yes.,"{'id': 'chatcmpl-79xJH6PI9x7xAN7w7f2QMLVMLxaJN', 'object': 'c

In [99]:
# get the number of rows were the chatbot respionse is not empty
df[df['chatbot_response'] != ''].shape

(100, 30)

In [86]:
df.tail()

,Unnamed: 0.1,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,...,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,Aaron,chatbot_response,chatbot_response_raw,chatbot_question
95,375,2719,90496399,NaN,G.Skill Trident Z Neo 16GB (2x8GB) 3600MHz DDR4 Desktop Memory Kit,NaN,169.00,AUD,NaN,2024334,...,1,False,"[-4.3931255341, 4.6727685928]",1,[1.0],1,1.0,"Yes,","{'id': 'chatcmpl-79xF60K6kQQSvK9krdJQNO4huvOWB', 'object': 'chat.completion', 'created': 1682607036, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 96, 'completion_tokens': 2, 'total_tokens': 98}, 'choices': [{'message': { ""content"": ""Yes,"", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: G.Skill Trident Z Neo 16GB (2x8GB) 3600MHz DDR4 Desktop Memory Kit, product 2: G.SKILL 16GB (8GBX2) 3600 MHZ DDR4 TRIDENT Z NEO RGB RAM (F4-3600C16D-16GTZNC)'}]"
96,377,3248,39143149,NaN,Shimano Ultegra CS-R8000 11sp Cassette 11-28T,"Shimano, Ultegra CS-R8000, 11sp. Cassette, 11-28T6 largest cogs spread on 2 aluminum spidersLight weight and high performanceAluminum lockringNickel plated finish11-34 cassette is compatible with MTB 10 speed and road 11 speed freehub bodies (use spacer for 11 speed road)Other sizes are compatib...",164.99,CAD,NaN,895823,...,0,True,"[3.8170776367, -4.2780303955]",0,[0.0],0,1.0,No.,"{'id': 'chatcmpl-79xF65YGw6JJoWH3Mgx9h3Ar6bI9g', 'object': 'chat.completion', 'created': 1682607036, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 67, 'completion_tokens': 2, 'total_tokens': 69}, 'choices': [{'message': { ""content"": ""No."", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: Shimano Ultegra CS-R8000 11sp Cassette 11-28T, product 2: Shimano Dura-Ace CS-R9100 Cassette 11 vitesses'}]"
97,378,2164,95352771,Logitech,Logitech C310 Webcam - 1 Megapixel - USB 2.0,"5 Megapixel Interpolated - 1280 x 720 Video - CMOS Sensor - Widescreen - Microphone - Monitor, Notebook",45.49,GBP,NaN,301204,...,1,False,"[-4.3837895393, 4.6717839241]",1,[1.0],1,1.0,Yes.,"{'id': 'chatcmpl-79xF7MhfrriOBkLvjXLi9iZN5RJV4', 'object': 'chat.completion', 'created': 1682607037, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 66, 'completion_tokens': 2, 'total_tokens': 68}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: Logitech C310 Webcam - 1 Megapixel - USB 2.0, product 2: Logitech C310 Siyah HD Webcam (960-001065 V-U0015)'}]"
98,379,35,42121103,NaN,T1204171704100 Tissot Seastar 1000 Quartz,"The Tissot Seastar 1000 merges style and performance without compromising either. The diving inspiration shapes both the appearance and the functionality of this watch. It maintains its performance to a pressure of 30 bar (300m/1'000ft), combining underwater sports and a preference for a sophist...",451.0,EUR,NaN,1167881,...,0,True,"[-1.9376914501, 1.9234960079]",1,[0.0],0,1.0,Yes.,"{'id': 'chatcmpl-79xF8USaNC1lA1BZKRecC8aCBBdNL', 'object': 'chat.completion', 'created': 1682607038, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 67, 'completion_tokens': 2, 'total_tokens': 69}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: T1204171704100 Tissot Seastar 1000 Quartz, product 2: Tissot Seastar 1000 T120.417.11.051.00'}]"
99,392,2822,4225266,NaN,Corsair Vengeance RGB Pro 16GB 2x8GB 288-Pin DDR4 DRAM DDR4 3000 (PC4 24000) Desktop Memory Model - White | CMW16GX4M2C3000C15W,DDR4 3000 (PC4 24000

In [87]:
def clean_response(response):
    if "yes" in response.lower():
        return 1
    elif "no" in response.lower():
        return 0
    else:
        return -1

In [88]:
# add a coloumn with a clean version of the chatbot response if the chatresponse contains Yes/yes it will be 1 else if it contains No/no it will be 0 if it is 0 or 1 
df['chatbot_response_clean'] = df['chatbot_response'].apply(lambda x: clean_response(x))

In [89]:
# get the current date and time
now = datetime.now()

# save the dataframe as a json file
df.to_json(f'data/results/chat_gpt/curated/{now}_simple_promt.json')

In [90]:
# look at the wrong answers
df[df['chatbot_response_clean'] != df['label']].head()

,Unnamed: 0.1,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,...,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,Aaron,chatbot_response,chatbot_response_raw,chatbot_question,chatbot_response_clean
0,0,1349,45334125,NaN,Pen Drive SanDisk 64GB Cruzer Glide USB 2.0,NaN,"9,86",EUR,NaN,763994,...,True,"[3.7204918861, -4.1918787956]",0,[0.0],0,1.0,Yes.,"{'id': 'chatcmpl-79x5wcz4L3RtfrJHWiVbaw7fi7N06', 'object': 'chat.completion', 'created': 1682606468, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 70, 'completion_tokens': 2, 'total_tokens': 72}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: Pen Drive SanDisk 64GB Cruzer Glide USB 2.0, product 2: SanDisk 64GB SDIX30N-064G-GN6NN USB 3.0 Flash Drive'}]",1
5,7,2955,53025257,NaN,Ubiquiti UVC-G3-Bullet UniFi IP Bullet Camera,NaN,$‚Äé197.95,USD,NaN,591042,...,True,"[-4.3029179573, 4.5916333199]",1,[0.0],0,1.0,Yes,"{'id': 'chatcmpl-79x6CUGWDTGFWe58ZJ6a3SAeuNuYZ', 'object': 'chat.completion', 'created': 1682606484, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 58, 'completion_tokens': 1, 'total_tokens': 59}, 'choices': [{'message': { ""content"": ""Yes"", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: Ubiquiti UVC-G3-Bullet UniFi IP Bullet Camera, product 2: UniFi Video Camera G3 IR 1080 HD Bullet'}]",1
8,14,1847,71378693,Logitech,Logitech H390 Stereo USB PC Computer Headset,Buy Logitech H390 Stereo USB PC Computer Headsets - MPN: 981-000406.A Stereo USB Headset For PC & MAC,45.59,GBP,NaN,149318,...,True,"[3.8143596649, -4.2751893997]",0,[0.0],0,1.0,Yes.,"{'id': 'chatcmpl-79x6E66ThTLM1gSX9113tHGdHYf0s', 'object': 'chat.completion', 'created': 1682606486, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 54, 'completion_tokens': 2, 'total_tokens': 56}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: Logitech H390 Stereo USB PC Computer Headset, product 2: LOGITECH - USB HEADSET H540 IN ACCS'}]",1
9,16,7,66462363,Audio Technica,Audio Technica ATH-M50x Professional Monitor Headphones,"This is the most critically acclaimed model in the M-Series line, praised by top audio engineers and pro audio reviewers year after year.",122.00,GBP,NaN,1889634,...,True,"[-1.9811097383, 1.9827538729]",1,[0.0],0,1.0,Yes,"{'id': 'chatcmpl-79x6F6jxGHNlMQFJELNke9pInNnGP', 'object': 'chat.completion', 'created': 1682606487, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 58, 'completion_tokens': 1, 'total_tokens': 59}, 'choices': [{'message': { ""content"": ""Yes"", ""role"": ""assistant"" }, 'finish_reason': ...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: Audio Technica ATH-M50x Professional Monitor Headphones, product 2: Audio Technica ATH-M50x Professional Monitor Audiophile Headphones - White'}]",1
21,54,2437,55529309,NaN,"1 TB WD My Passport, 2.5\"", USB 3.0, WD Backup - R√∂d",NaN,849.00,SEK,NaN,2807367,...,True,"[-1.7298119068000002, 1.6950979233]",1,[0.0],0,1.0,Yes.,"{'id': 'chatcmpl-79xCsFyMolBJtMdRBgfk0TU8fNXNb', 'object': 'chat.completion', 'created': 1682606898, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 95, 'completion_tokens': 2, 'total_tokens': 97}, 'choices': [{'message': { ""content"": ""Yes."", ""role"": ""assistant"" }, 'finish_reason':...","[{'role': 'user', 'content': 'Do the following two product decriptions match. Answer with yes or no. product 1: 1 TB WD My Passport, 2.5\"", USB 3.0, WD Backup - R√∂d, pr

In [91]:
# Check the accuracy of the chatbot 
df[df['chatbot_response_clean'] == df['label']].shape[0]
print(f"Accuracy: {df[df['chatbot_response_clean'] == df['label']].shape[0] / df.shape[0]}")

Accuracy: 0.75


In [92]:
# Check the number of -1 responses
df[df['chatbot_response_clean'] == -1].shape[0]

0

In [93]:
# Look at 5 complete chatbot responses dont limit the output length
# set the max_rows and max_columns options to None
# set the max_colwidth option to None
pd.set_option('display.max_colwidth', 300)

# display the first 5 chatbot responses where chatbot_response_raw is not -1
df['chatbot_response_raw'].head(3)

0    {'id': 'chatcmpl-79x5wcz4L3RtfrJHWiVbaw7fi7N06', 'object': 'chat.completion', 'created': 1682606468, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 70, 'completion_tokens': 2, 'total_tokens': 72}, 'choices': [{'message': {
  "content": "Yes.",
  "role": "assistant"
}, 'finish_reason':...
1    {'id': 'chatcmpl-79x5whTwLODy9xxgyhBZlaM7rtxpb', 'object': 'chat.completion', 'created': 1682606468, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 87, 'completion_tokens': 2, 'total_tokens': 89}, 'choices': [{'message': {
  "content": "Yes.",
  "role": "assistant"
}, 'finish_reason':...
2    {'id': 'chatcmpl-79x5xL7wIQrrL0JzXBthsJHUhSdRZ', 'object': 'chat.completion', 'created': 1682606469, 'model': 'gpt-3.5-turbo-0301', 'usage': {'prompt_tokens': 69, 'completion_tokens': 2, 'total_tokens': 71}, 'choices': [{'message': {
  "content": "No.",
  "role": "assistant"
}, 'finish_reason': ...
Name: chatbot_response_raw, dtype: object

In [94]:
# Lets look at all mistakes only show the title_left, title_right, label and chatbot_response_clean
df[df['chatbot_response_clean'] != df['label']][['title_left', 'title_right', 'label', 'chatbot_response_clean']]


,title_left,title_right,label,chatbot_response_clean
0,Pen Drive SanDisk 64GB Cruzer Glide USB 2.0,SanDisk 64GB SDIX30N-064G-GN6NN USB 3.0 Flash Drive,0,1
5,Ubiquiti UVC-G3-Bullet UniFi IP Bullet Camera,UniFi Video Camera G3 IR 1080 HD Bullet,0,1
8,Logitech H390 Stereo USB PC Computer Headset,LOGITECH - USB HEADSET H540 IN ACCS,0,1
9,Audio Technica ATH-M50x Professional Monitor Headphones,Audio Technica ATH-M50x Professional Monitor Audiophile Headphones - White,0,1
21,"1 TB WD My Passport, 2.5\"", USB 3.0, WD Backup - R√∂d",WD Black 1TB My Passport Portable USB3.0 HDD (WDBYVG0010BBK-WESN) (Warranty 3years with WD SG),0,1
24,Apple Airpods Pro with Wireless Charging Case,"Apple AirPods (2nd Generation) Bluetooth Earbuds w/ Wireless Charging Case, White (MRXJ2AM/A)",0,1
25,HIKVISION 16CH TURBO DVR 16IP & 16 BNC,Hikvision 16CH Turbo Full HD1080P DVR,0,1
28,CASTI LOGITECH UC Corded Stereo USB Headset H570e (Leatherette Pad) - Business EMEA,LOGITECH - USB Headset H570e Mono USB EMEA,1,0
31,CORSAIR Crystal Series 570X RGB Mid Tower,Corsair Crystal Series 570X RGB White ATX,0,1
33,Apple AirPods with Charging Case / COMPARE ON Amazon $139,Apple AirPods with Charging Case - (Brand New),0,1


In [95]:
# how many currencies are there in the dataset
df['priceCurrency_left'].unique()

array(['EUR', 'CZK', 'USD', 'GBP', 'AUD', 'CAD', 'SEK', nan, 'KYD', 'RON',
       'PLN', 'SGD', 'SAR', 'DKK', 'AED', 'eur', 'NOK', 'RUB', 'ZAR',
       'MUR'], dtype=object)

In [96]:
# How often does priceCurrency_left and priceCurrency_right match
df[df['priceCurrency_left'] == df['priceCurrency_right']].shape[0]

18